In [1]:
!pip install transformers torch pillow sentence-transformers langchain langchain-community pypdf faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from PIL import Image
import torch
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

class FreeStructuralDefectAnalyzer:
    def __init__(self, pdf_path=None):
        self.pdf_path = pdf_path
        self.vectorstore = None
        self.image_captioner = None
        self.vqa_model = None
        self.classifier = None

    def setup(self):
        """Setup all models"""
        print("=" * 80)
        print("INITIALIZING FREE STRUCTURAL ANALYSIS SYSTEM")
        print("=" * 80)

        # 1. Image Captioning Model
        print("\n[1/4] Loading image captioning model (BLIP)...")
        self.blip_processor = BlipProcessor.from_pretrained(
            "Salesforce/blip-image-captioning-large"
        )
        self.blip_model = BlipForConditionalGeneration.from_pretrained(
            "Salesforce/blip-image-captioning-large"
        )
        print("      ✓ BLIP model loaded")

        # 2. Visual Question Answering Model
        print("\n[2/4] Loading Visual QA model (BLIP-VQA)...")
        self.vqa_processor = BlipProcessor.from_pretrained(
            "Salesforce/blip-vqa-base"
        )
        self.vqa_model = BlipForConditionalGeneration.from_pretrained(
            "Salesforce/blip-vqa-base"
        )
        print("      ✓ VQA model loaded")

        # 3. Text Classification for defect analysis
        print("\n[3/4] Loading zero-shot classifier...")
        self.classifier = pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli"
        )
        print("      ✓ Classifier loaded")

        # 4. Load PDF guidelines (optional)
        if self.pdf_path:
            print("\n[4/4] Loading building guidelines PDF...")
            loader = PyPDFLoader(self.pdf_path)
            documents = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            texts = text_splitter.split_documents(documents)

            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
            self.vectorstore = FAISS.from_documents(texts, embeddings)
            print(f"      ✓ Loaded {len(documents)} pages from PDF")

        print("\n" + "=" * 80)
        print("✅ SYSTEM READY!")
        print("=" * 80 + "\n")

    def analyze_image(self, image_path):
        """Generate detailed image description"""
        image = Image.open(image_path).convert('RGB')

        # General caption
        inputs = self.blip_processor(image, return_tensors="pt")
        out = self.blip_model.generate(**inputs, max_length=100)
        general_caption = self.blip_processor.decode(out[0], skip_special_tokens=True)

        # Detailed technical caption
        text_prompt = "a detailed view of"
        inputs = self.blip_processor(image, text_prompt, return_tensors="pt")
        out = self.blip_model.generate(**inputs, max_length=100)
        detailed_caption = self.blip_processor.decode(out[0], skip_special_tokens=True)

        return general_caption, detailed_caption

    def ask_visual_questions(self, image_path):
        """Ask specific questions about the image"""
        image = Image.open(image_path).convert('RGB')

        questions = [
            "Is this a crack?",
            "Is the crack diagonal or horizontal?",
            "Is there exposed rebar?",
            "Is there concrete spalling?",
            "Is this on a column or beam?",
            "Is the crack wide?",
            "Is this structural damage?"
        ]

        answers = {}
        for question in questions:
            inputs = self.vqa_processor(image, question, return_tensors="pt")
            out = self.vqa_model.generate(**inputs, max_length=20)
            answer = self.vqa_processor.decode(out[0], skip_special_tokens=True)
            answers[question] = answer

        return answers

    def classify_defect(self, description, vqa_answers):
        """Classify as structural or cosmetic"""

        # Combine all information
        full_description = f"{description}. "
        for q, a in vqa_answers.items():
            full_description += f"{q} {a}. "

        # Define categories
        candidate_labels = [
            "structural shear crack",
            "structural flexural crack",
            "structural column damage",
            "concrete spalling with rebar exposure",
            "short column defect",
            "cosmetic plaster crack",
            "cosmetic paint damage",
            "cosmetic surface issue"
        ]

        # Classify
        result = self.classifier(
            full_description,
            candidate_labels,
            multi_label=False
        )

        # Determine if structural or cosmetic
        top_label = result['labels'][0]
        top_score = result['scores'][0]

        is_structural = 'structural' in top_label.lower()
        classification = "STRUCTURAL" if is_structural else "COSMETIC"

        # Severity based on keywords
        severity = "LOW"
        if is_structural:
            if any(kw in full_description.lower() for kw in ['rebar', 'spalling', 'wide', 'column']):
                severity = "CRITICAL"
            elif 'shear' in top_label or 'diagonal' in full_description.lower():
                severity = "HIGH"
            else:
                severity = "MEDIUM"

        return {
            'classification': classification,
            'predicted_defect': top_label,
            'confidence': top_score,
            'severity': severity,
            'all_predictions': list(zip(result['labels'], result['scores']))
        }

    def analyze_structural_defect(self, image_path):
        """Complete structural defect analysis"""
        print("\n" + "=" * 80)
        print("STRUCTURAL DEFECT ANALYSIS")
        print("=" * 80)
        print(f"Image: {image_path}\n")

        # Step 1: Analyze image
        print("🔍 Step 1: Analyzing image...")
        general, detailed = self.analyze_image(image_path)
        print(f"   General: {general}")
        print(f"   Detailed: {detailed}")

        # Step 2: Visual Q&A
        print("\n🔍 Step 2: Visual question answering...")
        vqa_answers = self.ask_visual_questions(image_path)
        for q, a in vqa_answers.items():
            print(f"   Q: {q}")
            print(f"   A: {a}")

        # Step 3: Classification
        print("\n🔍 Step 3: Classifying defect type...")
        classification = self.classify_defect(detailed, vqa_answers)

        # Display results
        print("\n" + "=" * 80)
        print("📊 ANALYSIS RESULTS")
        print("=" * 80)
        print(f"\n🏷️  CLASSIFICATION: {classification['classification']}")
        print(f"⚠️  SEVERITY LEVEL: {classification['severity']}")
        print(f"🎯 PREDICTED DEFECT: {classification['predicted_defect']}")
        print(f"📈 CONFIDENCE: {classification['confidence']:.2%}")

        print("\n📋 DETAILED PREDICTIONS:")
        print("-" * 80)
        for label, score in classification['all_predictions'][:5]:
            indicator = "🔴" if "structural" in label.lower() else "🟢"
            print(f"{indicator} {score:.2%} - {label}")

        # Recommendations
        print("\n💡 RECOMMENDATIONS:")
        print("-" * 80)
        if classification['classification'] == "STRUCTURAL":
            if classification['severity'] == "CRITICAL":
                print("⚠️  URGENT ACTION REQUIRED:")
                print("   - Immediate structural engineer inspection")
                print("   - Restrict access to affected area")
                print("   - Document and monitor closely")
                print("   - Consider temporary shoring")
            elif classification['severity'] == "HIGH":
                print("⚠️  PRIORITY REPAIR NEEDED:")
                print("   - Schedule structural engineer inspection within 48 hours")
                print("   - Monitor for progression")
                print("   - Avoid heavy loading in affected area")
            else:
                print("⚠️  INSPECTION RECOMMENDED:")
                print("   - Schedule structural assessment")
                print("   - Monitor and document")
                print("   - Plan for repair")
        else:
            print("✅ COSMETIC ISSUE:")
            print("   - Schedule routine maintenance")
            print("   - Cosmetic repair sufficient")
            print("   - Continue normal monitoring")

        # Building code references
        if self.vectorstore:
            print("\n📚 RELEVANT BUILDING CODE SECTIONS:")
            print("=" * 80)
            search_query = f"{classification['predicted_defect']} crack repair structural requirements"
            docs = self.vectorstore.similarity_search(search_query, k=2)
            for i, doc in enumerate(docs, 1):
                print(f"\n[Reference {i} - Page {doc.metadata.get('page', 'N/A')}]")
                print("-" * 80)
                print(doc.page_content[:500] + "...")

        print("\n" + "=" * 80 + "\n")

        return classification

    def batch_analyze(self, image_paths):
        """Analyze multiple images"""
        results = []

        for i, img_path in enumerate(image_paths, 1):
            print(f"\n{'#'*80}")
            print(f"IMAGE {i}/{len(image_paths)}")
            print(f"{'#'*80}")

            result = self.analyze_structural_defect(img_path)
            results.append({
                'image': img_path,
                'result': result
            })

        # Summary report
        print("\n" + "=" * 80)
        print("BATCH ANALYSIS SUMMARY REPORT")
        print("=" * 80)

        structural = [r for r in results if r['result']['classification'] == 'STRUCTURAL']
        cosmetic = [r for r in results if r['result']['classification'] == 'COSMETIC']

        critical = [r for r in structural if r['result']['severity'] == 'CRITICAL']
        high = [r for r in structural if r['result']['severity'] == 'HIGH']

        print(f"\n📊 OVERVIEW:")
        print(f"   Total Images Analyzed: {len(results)}")
        print(f"   Structural Issues: {len(structural)}")
        print(f"   Cosmetic Issues: {len(cosmetic)}")

        print(f"\n⚠️  SEVERITY BREAKDOWN:")
        print(f"   Critical: {len(critical)}")
        print(f"   High: {len(high)}")
        print(f"   Medium/Low: {len(structural) - len(critical) - len(high)}")

        if critical:
            print(f"\n🚨 CRITICAL ISSUES REQUIRING IMMEDIATE ATTENTION:")
            for r in critical:
                print(f"   - {r['image']}: {r['result']['predicted_defect']}")

        if high:
            print(f"\n⚠️  HIGH PRIORITY ISSUES:")
            for r in high:
                print(f"   - {r['image']}: {r['result']['predicted_defect']}")

        print("\n" + "=" * 80 + "\n")

        return results

# ============================================================================
# USAGE EXAMPLES
# ============================================================================

# Initialize the system
analyzer = FreeStructuralDefectAnalyzer(
    pdf_path=None  # Optional, set to None to skip PDF loading if file is not available
)
analyzer.setup()

# Example 1: Analyze single image
# Make sure to replace '/content/crack_photo.jpg' with a valid image path or upload the image.
# For now, we'll comment out the analysis to prevent further file-not-found errors if images are missing.
# result = analyzer.analyze_structural_defect("/content/crack_photo.jpg")

# Example 2: Batch analysis
# Make sure to replace '/content/crack1.jpg', etc., with valid image paths or upload the images.
# For now, we'll comment out the analysis to prevent further file-not-found errors if images are missing.
# results = analyzer.batch_analyze([
#     "/content/crack1.jpg",
#     "/content/crack2.jpg",
#     "/content/crack3.jpg"
# ])

INITIALIZING FREE STRUCTURAL ANALYSIS SYSTEM

[1/4] Loading image captioning model (BLIP)...


Loading weights:   0%|          | 0/616 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-image-captioning-large
Key                                       | Status     |  | 
------------------------------------------+------------+--+-
text_decoder.bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


      ✓ BLIP model loaded

[2/4] Loading Visual QA model (BLIP-VQA)...


Loading weights:   0%|          | 0/472 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-vqa-base
Key                                                                        | Status     |  | 
---------------------------------------------------------------------------+------------+--+-
text_encoder.encoder.layer.{0...11}.output.LayerNorm.bias                  | UNEXPECTED |  | 
text_encoder.encoder.layer.{0...11}.output.dense.weight                    | UNEXPECTED |  | 
text_encoder.encoder.layer.{0...11}.output.dense.bias                      | UNEXPECTED |  | 
text_encoder.encoder.layer.{0...11}.attention.self.value.bias              | UNEXPECTED |  | 
text_encoder.encoder.layer.{0...11}.intermed

      ✓ VQA model loaded

[3/4] Loading zero-shot classifier...


Loading weights:   0%|          | 0/515 [00:00<?, ?it/s]

      ✓ Classifier loaded

✅ SYSTEM READY!



In [5]:
# Upload images in Colab
from google.colab import files

print("📸 Upload your crack/defect images:")
uploaded = files.upload()

# Analyze all uploaded images
image_paths = list(uploaded.keys())
analyzer.batch_analyze(image_paths)

📸 Upload your crack/defect images:


Saving beam crack 01.jfif to beam crack 01.jfif

################################################################################
IMAGE 1/1
################################################################################

STRUCTURAL DEFECT ANALYSIS
Image: beam crack 01.jfif

🔍 Step 1: Analyzing image...
   General: a close up of a crack in the wall of a bathroom
   Detailed: a detailed view of a crack in the floor of a bathroom

🔍 Step 2: Visual question answering...
   Q: Is this a crack?
   A: is this a crack?
   Q: Is the crack diagonal or horizontal?
   A: is the crack diagonal or horizontal?
   Q: Is there exposed rebar?
   A: is there exposed rebar?
   Q: Is there concrete spalling?
   A: is there concrete spalling?
   Q: Is this on a column or beam?
   A: is this on a column or beam?
   Q: Is the crack wide?
   A: is the crack wide?
   Q: Is this structural damage?
   A: is this structural damage?

🔍 Step 3: Classifying defect type...

📊 ANALYSIS RESULTS

🏷️  CLASSIFICATION: COS

[{'image': 'beam crack 01.jfif',
  'result': {'classification': 'COSMETIC',
   'predicted_defect': 'concrete spalling with rebar exposure',
   'confidence': 0.7111527323722839,
   'severity': 'LOW',
   'all_predictions': [('concrete spalling with rebar exposure',
     0.7111527323722839),
    ('structural column damage', 0.1300022006034851),
    ('structural flexural crack', 0.05400834232568741),
    ('structural shear crack', 0.040456004440784454),
    ('short column defect', 0.0363411009311676),
    ('cosmetic surface issue', 0.0168857853859663),
    ('cosmetic plaster crack', 0.0075498525984585285),
    ('cosmetic paint damage', 0.0036039361730217934)]}}]

In [6]:
# Upload images in Colab
from google.colab import files

print("📸 Upload your crack/defect images:")
uploaded = files.upload()

# Analyze all uploaded images
image_paths = list(uploaded.keys())
analyzer.batch_analyze(image_paths)


📸 Upload your crack/defect images:


Saving Beam crack 02.jfif to Beam crack 02.jfif

################################################################################
IMAGE 1/1
################################################################################

STRUCTURAL DEFECT ANALYSIS
Image: Beam crack 02.jfif

🔍 Step 1: Analyzing image...
   General: there is a picture of a room with a ceiling that has been stripped
   Detailed: a detailed view of a ceiling with a hole in the ceiling

🔍 Step 2: Visual question answering...
   Q: Is this a crack?
   A: is this a crack?
   Q: Is the crack diagonal or horizontal?
   A: is the crack diagonal or horizontal?
   Q: Is there exposed rebar?
   A: is there exposed rebar?
   Q: Is there concrete spalling?
   A: is there concrete spalling?
   Q: Is this on a column or beam?
   A: is this on a column or beam?
   Q: Is the crack wide?
   A: is the crack wide?
   Q: Is this structural damage?
   A: is this structural damage?

🔍 Step 3: Classifying defect type...

📊 ANALYSIS RESULTS

🏷️

[{'image': 'Beam crack 02.jfif',
  'result': {'classification': 'COSMETIC',
   'predicted_defect': 'concrete spalling with rebar exposure',
   'confidence': 0.742529571056366,
   'severity': 'LOW',
   'all_predictions': [('concrete spalling with rebar exposure',
     0.742529571056366),
    ('structural column damage', 0.12477575242519379),
    ('structural flexural crack', 0.04245448112487793),
    ('structural shear crack', 0.03583596274256706),
    ('short column defect', 0.032543476670980453),
    ('cosmetic surface issue', 0.011910424567759037),
    ('cosmetic plaster crack', 0.005926030687987804),
    ('cosmetic paint damage', 0.004024311900138855)]}}]

In [8]:
# Upload images in Colab
from google.colab import files

print("📸 Upload your crack/defect images:")
uploaded = files.upload()

# Analyze all uploaded images
image_paths = list(uploaded.keys())
analyzer.batch_analyze(image_paths)


📸 Upload your crack/defect images:


Saving col ceack 3.jfif to col ceack 3.jfif

################################################################################
IMAGE 1/1
################################################################################

STRUCTURAL DEFECT ANALYSIS
Image: col ceack 3.jfif

🔍 Step 1: Analyzing image...
   General: arafed concrete column with a hole in it with a sign on it
   Detailed: a detailed view of a concrete pillar with a hole in it

🔍 Step 2: Visual question answering...
   Q: Is this a crack?
   A: is this a crack?
   Q: Is the crack diagonal or horizontal?
   A: is the crack diagonal or horizontal?
   Q: Is there exposed rebar?
   A: is there exposed rebar?
   Q: Is there concrete spalling?
   A: is there concrete spalling?
   Q: Is this on a column or beam?
   A: is this on a column or beam?
   Q: Is the crack wide?
   A: is the crack wide?
   Q: Is this structural damage?
   A: is this structural damage?

🔍 Step 3: Classifying defect type...

📊 ANALYSIS RESULTS

🏷️  CLASSIFICATIO

[{'image': 'col ceack 3.jfif',
  'result': {'classification': 'COSMETIC',
   'predicted_defect': 'concrete spalling with rebar exposure',
   'confidence': 0.659702718257904,
   'severity': 'LOW',
   'all_predictions': [('concrete spalling with rebar exposure',
     0.659702718257904),
    ('structural column damage', 0.255109041929245),
    ('structural flexural crack', 0.02855474315583706),
    ('short column defect', 0.02432461455464363),
    ('structural shear crack', 0.020513826981186867),
    ('cosmetic surface issue', 0.0070011853240430355),
    ('cosmetic plaster crack', 0.0027464476879686117),
    ('cosmetic paint damage', 0.0020474521443247795)]}}]

In [7]:
# Upload images in Colab
from google.colab import files

print("📸 Upload your crack/defect images:")
uploaded = files.upload()

# Analyze all uploaded images
image_paths = list(uploaded.keys())
analyzer.batch_analyze(image_paths)

📸 Upload your crack/defect images:


Saving col crack 01.jfif to col crack 01.jfif

################################################################################
IMAGE 1/1
################################################################################

STRUCTURAL DEFECT ANALYSIS
Image: col crack 01.jfif

🔍 Step 1: Analyzing image...
   General: three pieces of stone with a carving of a man and a woman
   Detailed: a detailed view of a group of four stone sculptures on a white surface

🔍 Step 2: Visual question answering...
   Q: Is this a crack?
   A: is this a crack?
   Q: Is the crack diagonal or horizontal?
   A: is the crack diagonal or horizontal?
   Q: Is there exposed rebar?
   A: is there exposed rebar?
   Q: Is there concrete spalling?
   A: is there concrete spalling?
   Q: Is this on a column or beam?
   A: is this on a column or beam?
   Q: Is the crack wide?
   A: is the crack wide?
   Q: Is this structural damage?
   A: is this structural damage?

🔍 Step 3: Classifying defect type...

📊 ANALYSIS RESULTS


[{'image': 'col crack 01.jfif',
  'result': {'classification': 'COSMETIC',
   'predicted_defect': 'concrete spalling with rebar exposure',
   'confidence': 0.6482559442520142,
   'severity': 'LOW',
   'all_predictions': [('concrete spalling with rebar exposure',
     0.6482559442520142),
    ('structural column damage', 0.1671174168586731),
    ('structural flexural crack', 0.05535547062754631),
    ('short column defect', 0.04753004387021065),
    ('structural shear crack', 0.04175886884331703),
    ('cosmetic surface issue', 0.0233489777892828),
    ('cosmetic plaster crack', 0.009973129257559776),
    ('cosmetic paint damage', 0.006660132203251123)]}}]